In [ ]:
import numpy as np
import pandas as pd 
from scipy.stats import binom
from scipy.stats import bernoulli
from scipy.stats import norm
from scipy.stats import binom_test

In [ ]:
ctl = pd.read_excel("./Final Project Results.xlsx",sheetname=0)
exp = pd.read_excel("./Final Project Results.xlsx",sheetname=1)
ctl

In [ ]:
## readin data 
ctl = pd.read_excel("./Final Project Results.xlsx",sheetname=0)
exp = pd.read_excel("./Final Project Results.xlsx",sheetname=1)


## data wrangle
ctl["Click through"] = ctl["Clicks"] / ctl["Pageviews"]
exp["Click through"] = exp["Clicks"] / exp["Pageviews"]
ctl["Retention"] = ctl["Payments"] / ctl["Enrollments"]
exp["Retention"] = exp["Payments"] / exp["Enrollments"]
ctl["Gross conversion"] = ctl["Enrollments"] / ctl["Clicks"]
exp["Gross conversion"] = exp["Enrollments"] / exp["Clicks"]
ctl["Net conversion"] = ctl["Payments"] / ctl["Clicks"]
exp["Net conversion"] = exp["Payments"] / exp["Clicks"]

In [ ]:
diff = (exp["Click through"] - ctl["Click through"])
print("CTP:",norm.interval(0.95,0,diff.std()))
lower,uper = norm.interval(0.95,0,diff.std())
# diff[(diff < lower && diff > uper)]

# print ("ob_CTP:", ctl["Click through"].mean())
# print ("-" *64)

In [ ]:
###sanity check
### Cookie
n_pv = exp["Pageviews"].sum() + ctl["Pageviews"].sum()
print ("pv:",np.array(binom.interval(0.95,n_pv,0.5))/n_pv)
print ("ob_pv:", ctl["Pageviews"].sum()/n_pv)
print ("-" *64)

### click
n_clk = exp["Clicks"].sum() + ctl["Clicks"].sum()
print ("clk:",np.array(binom.interval(0.95,n_clk,0.5))/n_clk)
print ("ob_clk:", ctl["Clicks"].sum()/n_clk)
print ("-" *64)

### CTR 
p_pool = (ctl["Clicks"].sum() + exp["Clicks"].sum()) / (ctl["Pageviews"].sum()+exp["Pageviews"].sum())
std_pool = np.sqrt(p_pool*(1-p_pool)*(1/ctl["Pageviews"].sum()+1/exp["Pageviews"].sum()))
print("ob_CTR:",exp["Click through"].mean())
print("CTR:",norm.interval(0.95,p_pool,std_pool))


In [ ]:
### effect test 
d_1 = (exp["Payments"].sum()/exp["Enrollments"].sum())-(ctl["Payments"].sum()/ctl["Enrollments"].sum())
d_2 = (exp["Payments"].sum()/exp["Clicks"].sum())-(ctl["Payments"].sum()/ctl["Clicks"].sum())
ese1 = (exp["Retention"]-ctl["Retention"]).std()
ese2 = exp["Net conversion"].std()-ctl["Net conversion"].std()

p_hat1 = (ctl["Payments"].sum()+exp["Payments"].sum())/(ctl["Enrollments"].sum()+exp["Enrollments"].sum())
p_hat2 = (ctl["Payments"].sum()+exp["Payments"].sum())/(ctl["Clicks"].sum()+exp["Clicks"].sum())
se1 = np.sqrt(p_hat1*(1-p_hat1)*(1/ctl["Enrollments"].sum()+1/exp["Enrollments"].sum()))
se2 = np.sqrt(p_hat2*(1-p_hat2)*(1/ctl["Clicks"].sum()+1/exp["Clicks"].sum()))

print (d_1)
print ("statis:", norm.interval(0.95,d_1,se1))
print (d_2)
print ("statis:", norm.interval(0.95,d_2,se2))

In [ ]:
### Retention sign test
print ((exp["Gross conversion"] - ctl["Gross conversion"] >0).sum())
print (binom_test(4,23,0.20625))

print ((exp["Retention"] - ctl["Retention"] >0).sum())
print (binom_test(13,23,0.53))

print ((exp["Net conversion"] - ctl["Net conversion"] >0).sum())
print (binom_test(10,23,0.1093125))